<h1>Explanation for script to characterize differential allelic representation in Chip seq peaks</h1>
<p>We will provide a python script with defined inputs and outputs that can be run from the command line. This notebook is
intended to help with developing and explaining the script.</p>

In [2]:
# This module allows us to run samtools in a separate process
import subprocess
from scipy.stats import binom

In [3]:
#In the script, the path to the BAM and BED files will be passed on the command line
#We need the corresponding FASTA file to show the reference base in column 3 of the pileup! Important!
bam_dir='/home/robinp/data/helium/'
bam_path=bam_dir + 'SRR7413398_1_sorted.bam'
bed_path=bam_dir + 'SRR7413398_1_sorted-Q-narrowPeak.bed'
fasta_dir='/home/robinp/data/ucsc/hg38/'
fasta_path=fasta_dir+ 'hg38.fa'

<h2>Running samtools mpileup</h2>
<p>The following subprocess call runs samtools with the pileup option. The output is identical as if we had started samtools from the command line.</p>

In [4]:
command=["samtools", "mpileup","--positions",bed_path,"-f",fasta_path,bam_path]
command=["samtools", "mpileup","-f",fasta_path,bam_path]
proc=subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,encoding="utf-8" )
output, err = proc.communicate()
#print (output) Do not uncomment for full pileup files!

<h2>mpileup format</h2>
<p>The mpileup format is defined in the online samtools documentation. We wrote an intuitive introduction in our computational
exome/genome analysis book, page 431. This Python script will read the lines of the pileup one by one and perform statistical analysis on them, and 
report the results to a file. I am including a few explanations of the pileup format here.</p>

my_chromosome	3	C	15	..........^F.^F.^F.^F.^F.	EEEEE>>>>>EEEEE
my_chromosome	44	G	130	.$.$.$.$.$.............................................................................................................................	BBBBBUUUUU_____aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

<p>The format is:
<ol>
<li>name of chromosome/reference</li>
<li>1-based position on chromosome</li>
<li>the reference base</p>
<li>The number of reads covering the base</li>
<li>The read bases</li>
<li>the qualities</li>
</ol>
</pre>
<p>Note that <tt>..........^F.^F.^F.^F.^F.</tt> means that all 15 bases were reference (the dot). Note that a comma means a reference match on the reverse strand. An upper-case character means there is a mismiatch on the forward strand and a lower case means a mismatch on the reverse strand.
According to the standard ^[c] means that the base was at the start of the read (where c is the mapping quality, e.g., ^F for quaoity F). </p>
<p>Similarly, $ means the base was at the end of the read. We can just ignore this.</p>
<p> + means an insertion between the reference base and the following reference base. The '+' is followed by a string to specify the inserted nucleotides (e.g., +4AGTG)</p>
<p>- means a deletiuon from the reference sequence. It is followed by a string to specify the deleted nucleotides, e.g., -4GTGT. </p>
<p>*, then it is a deleted base in a multiple base pair deletion that was previous specified using the - notation.</p>

In [ ]:
def process_pileup_line(ar):
    chrom=ar[0]
    pos=ar[1]
    refbase=ar[2]
    depth=int(ar[3])
    pup=ar[4]
    #ignore qualities
    ref_count=0
    alt_a=0
    alt_c=0
    alt_g=0
    alt_t=0
    deletion=0
    ins=0
    N=len(pup)
    i=0
    while i<N:
        c=pup[i]
        i=i+1 #move to next char
        if (c=='.' or c==','):
            ref_count=ref_count+1
        elif (c=='^'):
            i=i+1 #skip the next chararacter, which is the quality
        elif (c=='A' or c=='a'):
            alt_a=alt_a+1
        elif (c=='C' or c=='c'):
            alt_c=alt_c+1
        elif (c=='G' or c=='G'):
            alt_g=alt_g+1
        elif (c=='T' or c=='t'):
            alt_t=alt_t+1
        elif (c=='+'):
            deletion=deletion+1
            #the next character gives the length of the deletion string
            dellen_start=i
            while pup[i].isDigit():
                i=i+1
            delnum = pup[dellen_start,i]
            k=int(delnum)
            i=i+k
        elif (c=='i'):
            ins=ins+1
            #the next character gives the length of the deletion string
            dellen_start=i
            while pup[i].isDigit():
                i=i+1
            delnum = pup[dellen_start,i]
            k=int(delnum)
            i=i+k
    nonref_count=alt_a+alt_c+alt_g+alt_t+deletion+ins
    #print("depth",depth,"nonref",nonref_count)
    return [depth,ref_count,nonref_count,alt_a,alt_c,alt_g,alt_t,deletion,ins]
            
        

<h2>Statistical analysis</h2>
<p>We are interested in heterozygous variants whose frequency deviates significantly from 50% -- this could be an indication
that the binding of the transcription factor in ChIP-seq is altered by the variant. If the affinity is increased by
the variant, we expect more ALT reads; if the affinity is decreased, we expect more REF reads. If the affinity is not
affected, we expect 50-50%.</p>
<p>We will make a call for each position -- HOM-REF, HOM-ALT, HET-BAL, and HET-UNBAL. For HOM-REF, we calculate the probability of the data in the column using a probability of ALT reads of 2%, reflecting an assumed error probability of 2%. Similarlay, we calculate the proability of HOM-ALT using p=98%. For HET-BAL, we use p=50%. For HET-UNBAL, in principle the balance could be anywhere from 0-100%, but we will not be able to distinguish HOM-REF, HOM-ALT from 0% of 100%. Therefore, we make the assumption that 5%<=p<=40% and 60%<p<95%. 
\begin{equation*}
\hat{g} = \mathrm{argmax}_{g\in \left(\langle a,a\rangle,\langle a,b\rangle_{\mathrm{BAL}}, \langle a,b\rangle_{\mathrm{UNBAL}},\langle b,b\rangle\right) }p(g|D) 
\end{equation*}
<p>To calculate p for HET-BAL, and HET-UNBAL, we first calculate p_raw=nonref_count/depth. We then apply the following heuristic to calculate our final p</p>
\begin{equation*}
\hat{p} = \begin{cases}
\max (p_{raw},0.05) & p_{raw} \leq 0.4 \\
\min (p_{raw},0.95) & p_{raw} \geq 0.6 \\
\end{cases}
\end{equation*}
<p>If p_raw is between 40 and 60%, then we assume that the variant is HET-BAL and do not perform an explict comparison. Also, if the number of ALT calls is less than 2, we assume that the position is HOM-REF and do not perform an explicit comparison. TODO we should also consider a certain minimum depth. For now, I will implement 10.</p>

In [ ]:
def analyze_balance(depth,ref_count,nonref_count):
    if ref_count < 2:
        return "B"
    p=float(nonref_count)/float(depth)
    homref=binom.pmf(nonref_count,depth,0.02)
    homalt=binom.pmf(nonref_count,depth,0.98)
    hetbal=binom.pmf(nonref_count,depth,0.5)
    hetunbal=0 #default in case p<60% and p>40%
    if (p<=0.4 and p>=0.05):
        p=max(0.05,p)
        hetunbal=binom.pmf(nonref_count,depth,p)
    elif (p>=0.6 and p<=0.95):
        p=min(0.95,p)
        hetunbal=binom.pmf(nonref_count,depth,p)
    maxhom=max(homref,homalt) #dont care which hom
    maxhet=max(hetbal,hetunbal)
    #print("p",p,"homref",homref,"homalt",homalt,"hetbal",hetbal,"hetunbal",hetunbal)
    if maxhom>maxhet:
        return "HOM"
    elif hetbal>hetunbal:
        return "B"
    else:
        return "U"
        
        

In [ ]:
command=["samtools", "mpileup","-f",fasta_path,bam_path]
proc=subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,encoding="utf-8" )
#output, err = proc.communicate()
# THe first two lines should be skipped
#[mpileup] 1 samples in 1 input files
#<mpileup> Set max per-file depth to 8000
header=proc.stdout.readline() 
header=proc.stdout.readline()
while True:
    line = proc.stdout.readline().rstrip()
    if not line:
        break
    ar=line.split()
    #print (line)
    if (len(ar)<5):
        continue
    depth,ref_count,nonref_count,alt_a,alt_c,alt_g,alt_t,deletion,ins=process_pileup_line(ar)
    r=analyze_balance(depth,ref_count,nonref_count)
    print(ar[0],ar[1],r,depth,ref_count,nonref_count)